In [ ]:
import os
import numpy as np
import torch
from torch.export import export
import onnx
import tvm
from tvm import relax
from tvm.relax.frontend.torch import from_exported_program
from tvm.relax.frontend.onnx import from_onnx

In [ ]:
# Bước 1: Load model ONNX
onnx_model = onnx.load("MobileNetV2.onnx")

# Bước 2: Chuyển sang Relay
input_name = "input_2"   # Tên input trong model
input_shape = (1, 96, 96, 3)
shape_dict = {input_name: input_shape}

In [ ]:
# In tên input/output
print("Inputs:")
for inp in onnx_model.graph.input:
    print(" -", inp.name)

print("Outputs:")
for out in onnx_model.graph.output:
    print(" -", out.name)

In [ ]:
# Chuyển sang Relax IR
mod = from_onnx(
    onnx_model,
    shape_dict,
    keep_params_in_input=True  # True nếu muốn weight nằm ở input
)

In [ ]:
mod.show()

In [ ]:
# Nếu có file nhãn 1000 classes của ImageNet
with open("imagenet_classes.txt") as f:
    labels = [line.strip() for line in f]

In [ ]:
labels

In [ ]:
mod: tvm.IRModule = relax.transform.LegalizeOps()(mod)
mod.show()

In [ ]:
mod: tvm.IRModule = relax.get_pipeline("zero")(mod)
mod.show()

In [ ]:
target = tvm.target.Target("llvm -mtriple=aarch64-linux-gnu")
executable = relax.build(mod, target, exec_mode="compiled")
executable.export_library(file_name="MobileNetV2.so",
                          cc="/root/Programming/linux/customs/aarch64-linux-gnu/bin/aarch64-linux-gnu-g++",
                          options=["-nostdlib", "-shared", "-fPIC"])

In [ ]:
target = tvm.target.Target("llvm")
executable = relax.build(mod, target, exec_mode="compiled")
executable.export_library(file_name="MobileNetV2_native.so",
                          options=["-nostdlib", "-shared", "-fPIC"])

In [ ]:
# from tvm.runtime import save_param_dict, NDArray
# # params gốc từ detach_params
# params_nd = {k: v for k, v in params.items() if isinstance(v, NDArray)}

In [ ]:
# params["main"][-1].shape

In [ ]:
# params["main"].__len__()

In [ ]:
# # params trả về từ relax.frontend.detach_params(mod)
# main_params = params["main"]

# # Trường hợp main_params là list/tuple → convert sang dict
# # và lọc chỉ giữ NDArray
# param_dict = {}
# for i, v in enumerate(main_params):
#     param_dict[f"p{i}"] = v

# with open("vgg16_params.bin", "wb") as f:
#     f.write(save_param_dict(param_dict))

### Load model and weights

In [ ]:
dev = tvm.cpu()

In [ ]:
loaded_mod: tvm.runtime.Module = tvm.runtime.load_module("MobileNetV2_native.so")
vm = relax.VirtualMachine(loaded_mod, dev)

In [ ]:
# from tvm.runtime import load_param_dict
# # Đọc file nhị phân
# with open("vgg16_params.bin", "rb") as f:
#     param_bytes = f.read()

# # Giải nén trọng số
# params = load_param_dict(param_bytes)
# cpu_params = [params[k] for k in sorted(params.keys(), key=lambda k: int(k[1:]))]
# cpu_params.__len__()

In [ ]:
from PIL import Image

In [ ]:
# Input data
img = Image.open("DefectHole/SizeNotEn.bmp").convert("RGB")
img = img.resize((96, 96))
img_np  = np.array(img).astype("float32")

In [ ]:
# Chuyển sang dạng NCHW (batch=1)
# img_np = np.transpose(img_np, (2, 0, 1))  # (C, H, W)
cpu_data = np.expand_dims(img_np, 0)        # (1, C, H, W)

In [ ]:
cpu_data.tofile("data/SizeNotEn.bin")

In [ ]:
cpu_data = tvm.nd.array(cpu_data, dev)

In [ ]:
vm = relax.VirtualMachine(executable, dev)
cpu_out = vm["main"](cpu_data)

In [ ]:
scores = cpu_out.numpy().reshape(-1)
# Lấy class có xác suất cao nhất
pred_class = int(np.argmax(scores))
confidence = float(scores[pred_class])

print(f"Predicted index: {pred_class}")
print(f"Predicted label: {labels[pred_class]}")
print(f"Confidence: {confidence:.4f}")